In [1]:
import torch
import pandas as pd
import numpy as np
import os
from models import RGCN
import utils

C:\CondaEnvs\work3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test = pd.read_csv(os.path.join("test.csv"),header=None,sep='\t')
train = pd.read_csv(os.path.join("train.csv"),header=None,sep='\t')


In [3]:
test.shape

(11932, 3)

In [4]:
train.shape

(47727, 3)

In [5]:
triplets = pd.concat([train,test])
triplets.shape

(59659, 3)

In [6]:
triplets.head(10)

,0,1,2
0,DB00073,D008206,DB12240
1,DB00741,D011085,DB01234
2,DB06186,D011471,DB00958
3,DB00002,D006258,DB00958
4,DB06186,D009375,DB01611
5,DB01101,D005767,DB00112
6,DB11581,D015448,DB11703
7,DB05812,D009370,DB11901
8,DB06626,D007674,DB01268
9,DB00073,D007160,DB15140


In [7]:
drugs = pd.DataFrame(pd.concat([triplets[0],triplets[2]]).unique())

In [8]:
drugs.shape

(1105, 1)

In [9]:
type(drugs)

pandas.core.frame.DataFrame

In [10]:
drugs = drugs.head(100)

In [11]:
drugs

,0
0,DB00073
1,DB00741
2,DB06186
3,DB00002
4,DB01101
...,...
95,DB11730
96,DB00688
97,DB00695
98,DB00305


In [18]:
test_trip = []
for i in range(len(drugs)):
    for j in range(i+1,len(drugs)):
        tri = [i,2,j]
        test_trip.append(tri)
test_trip = pd.DataFrame(test_trip)
test_trip.shape

(4950, 3)

In [19]:
test_trip.head(10)

,0,1,2
0,0,2,1
1,0,2,2
2,0,2,3
3,0,2,4
4,0,2,5
5,0,2,6
6,0,2,7
7,0,2,8
8,0,2,9
9,0,2,10


In [21]:
src, rel, dst = np.array(test_trip).transpose()
uniq_entity, edges = np.unique((src, dst), return_inverse=True)
src, dst = np.reshape(edges, (2, -1))
relabeled_edges = np.stack((src, rel, dst)).transpose()


# Create bi-directional graph
src, dst = torch.cat((src, dst)), torch.cat((dst, src))
rel = torch.cat((rel, rel + num_rels))

edge_index = torch.stack((src, dst))
edge_type = rel

data = Data(edge_index = edge_index)
data.entity = torch.from_numpy(uniq_entity)
data.edge_type = edge_type
data.edge_norm = edge_normalization(edge_type, edge_index, len(uniq_entity), num_rels)
# data.samples = torch.from_numpy(samples)
# data.labels = torch.from_numpy(labels)

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [14]:
checkpoint = torch.load('best_mrr_model.pth')

In [16]:
model = RGCN(1105,1283,4,1)

In [17]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
model.eval()
entity_embedding = model(range(100), train_data.edge_index, train_data.edge_type, train_data.edge_norm)
